In [1]:
import os
import pandas as pd
import gradio as gr
import openai as OpenAI
from langchain_upstage import ChatUpstage
from langchain_upstage import UpstageEmbeddings
from langchain_core.messages import HumanMessage, SystemMessage 
from langchain.schema import Document
from langchain.chains import RetrievalQA
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.retrievers import EnsembleRetriever
from langchain.retrievers import MultiQueryRetriever
from langchain.retrievers import BM25Retriever
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import Predibase
from langchain.schema import Document
from typing import TypedDict
from langgraph.graph import END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig
from langchain_upstage import UpstageGroundednessCheck
from predibase import Predibase as pb
from kiwipiepy import Kiwi
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from googleapiclient.discovery import build
import requests

# langchain smith API KEY
LANGCHAIN_API_KEY="..."
LANGCHAIN_TRACING_V2='true'
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
LANGCHAIN_PROJECT="bisAI"

# Chat model API KEY
OPENAI_API_KEY = '...'
UPSTAGE_API_KEY = '...'
PREDIBASE_API_KEY = '...'

# Google Custom Search API KEY, ID
CSE_API_KEY = '...'
CSE_ID = '...'

os.environ["LANGCHAIN_TRACING_V2"] = LANGCHAIN_TRACING_V2
os.environ["LANGCHAIN_ENDPOINT"] = LANGCHAIN_ENDPOINT
os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT

os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["UPSTAGE_API_KEY"] = UPSTAGE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["PREDIBASE_API_KEY"] = PREDIBASE_API_KEY

kiwi = Kiwi()
upstage_ground_checker = UpstageGroundednessCheck()
OpenAIembedding = OpenAIEmbeddings()

# Data CSV file directory
sup_product_df = pd.read_csv("db/nutrient_data_final.csv")
med_product_df = pd.read_csv("db/new_medi.csv")
med_info_df = pd.read_csv("db/pdf_contents.csv")

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
/Users/lyw/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [3]:
def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]
  
def google_search(query, search_type='image', num_results=5):
    service = build("customsearch", "v1", developerKey=CSE_API_KEY)
    res = service.cse().list(
        q=query,
        cx=CSE_ID,
        searchType='image' if search_type == 'image' else None,  # 이미지 검색 여부
        num=num_results
    ).execute()
    return res['items']

def url_image_to_text(image_url):
  api_key = UPSTAGE_API_KEY  # UPSTAGE Document OCR API KEY
  image_url = image_url
  url = api_key
  headers = {"Authorization": f"Bearer {api_key}"}

  # Image Download
  image_response = requests.get(image_url)
  files = {"document": ("image.png", image_response.content, "image/png")}

  response = requests.post(url, headers=headers, files=files)
  return response.json()["text"]

def find_info_by_web_search(query):
  #query google image search
  query_image = google_search(query)
  additional_query_info = ""
  #Image to Text 
  for i in range(5):
    try:
      additional_query_info = url_image_to_text(query_image[i]['link'])
      break
    except:
      print('fail')
      continue
  return additional_query_info

def combine_sup_product(row):
    # 각 컬럼의 내용을 결합하여 하나의 텍스트로 만듦
    return f"제품명: {row['이름']}\n영양성분: {row['성분']}\n기능: {row['기능']}\n주의사항: {row['주의사항']}"
def combine_pdf(row):
    return f"제목: {row['제목']}\n요약: {row['요약']}\n내용: {row['내용']}"
def combine_med_product(row):
    return f"제품명: {row['제품명']}\n주성분: {row['주성분']}\n기능: {row['이 약의 효능은 무엇입니까?']}\n주의사항: {row['이 약의 사용상 주의사항은 무엇입니까?']}"

def retriever_medi(documents, embedding_model):
    vector_store = FAISS.from_documents(documents, embedding_model)
    faiss_retriever = vector_store.as_retriever(search_kwargs={"k": 1})
    kiwi_bm25_retriever = BM25Retriever.from_documents(documents, preprocess_func=kiwi_tokenize)
    
    kiwibm25_faiss_73 = EnsembleRetriever(
    retrievers=[kiwi_bm25_retriever, faiss_retriever],  # 사용할 검색 모델의 리스트
    weights=[0.7, 0.3],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
    )
    return faiss_retriever

def retriever_sup(documents, embedding_model):
    vector_store = FAISS.from_documents(documents, embedding_model)
    faiss_retriever = vector_store.as_retriever(search_kwargs={"k": 3})
    kiwi_bm25_retriever = BM25Retriever.from_documents(documents, preprocess_func=kiwi_tokenize)
    
    kiwibm25_faiss_73 = EnsembleRetriever(
    retrievers=[kiwi_bm25_retriever, faiss_retriever],
    weights=[0.7, 0.3],
    search_type="mmr",
    )
    return kiwibm25_faiss_73

In [4]:
documents_sup_product = [Document(page_content=combine_sup_product(row), metadata={"제품명": row['이름']}) for _, row in sup_product_df.iterrows()]
documents_medi_product = [Document(page_content=combine_med_product(row), metadata={"제품명": row['제품명']}) for _, row in med_product_df.iterrows()]
documents_medi_info = [Document(page_content=combine_pdf(row), metadata={"제목": row['제목']}) for _, row in med_info_df.iterrows()]

OpenAIembedding = OpenAIEmbeddings()
SolarEmbedding = UpstageEmbeddings(api_key=UPSTAGE_API_KEY, model="solar-embedding-1-large")

embedding_model = OpenAIembedding
#embedding_model = SolarEmbedding

kiwibm25_faiss_73_sup_product = retriever_sup(documents_sup_product, embedding_model)
kiwibm25_faiss_73_medi_product = retriever_medi(documents_medi_product, embedding_model)
kiwibm25_faiss_73_medi_info = retriever_medi(documents_medi_info, embedding_model)

In [5]:
retriever = kiwibm25_faiss_73_sup_product
query = "루테인 지아잔틴 아스타잔틴, 비타앤 히알루론산 피치맛"
docs = retriever.get_relevant_documents(query)
docs

/Users/lyw/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


[Document(metadata={'제품명': '루테인 지아잔틴 아스타잔틴'}, page_content='제품명: 루테인 지아잔틴 아스타잔틴\n영양성분: 비타민A,210ug|아스타잔틴,6|루테인지아잔틴복합추출물,20|\n기능: 시력 및 눈 피로감 케어|\n주의사항: * 영, 유아, 어린이, 임산부 및 수유부는 섭취에 주의|* 과다섭취 시 일시적으로 피부가 황색으로 변할 수 있음|* β-카로틴의 흡수를 저해할 수 있음|* 임산부와 수유부, 질병치료 중인 분은 섭취 전 의사와 상담 후 섭취하십시오.|* 특정 성분에 알레르기가 있으신 분은 원료명을 확인 후 섭취하십시오.|* 개봉 또는 섭취 시 포장재에 의해 상처를 입을 수 있으니 주의하시기 바랍니다.|'),
 Document(metadata={'제품명': '비타앤 히알루론산 피치맛'}, page_content='제품명: 비타앤 히알루론산 피치맛\n영양성분: 비타민C,300|히알루론산,120|\n기능: 항산화|피부 건강|\n주의사항: * 특이체질 및 알레르기 체질이신 경우 성분을 확인 하신 후 섭취하여 주시기 바랍니다.|* 포장지에 의해 상처를 입을수 있으니 주의 하시기 바랍니다.|* 유통기한이 경과한 제품은 섭취하지 마시기 바랍니다.|'),
 Document(metadata={'제품명': '루테인 지아잔틴 미니'}, page_content='제품명: 루테인 지아잔틴 미니\n영양성분: 루테인,18.182|지아잔틴,1.818|\n기능: 시력 및 눈 피로감 케어|\n주의사항: * 영·유아, 어린이, 임산부 및 수유부는 섭취에 주의|* 과다 섭취 시 일시적으로 피부가 황색으로 변할 수 있음|'),
 Document(metadata={'제품명': '루테인 지아잔틴 플러스'}, page_content='제품명: 루테인 지아잔틴 플러스\n영양성분: 비타민C,30|비타민E,3.3|아연,2.55|구리,240ug|루테인,18.182|지아잔틴,1.1818|\n기능: 항산화|면역력 증진|시력 및 눈 피로감 케어|\n주의사항: * 섭취

In [7]:
def prompt_classification(llm, prompt):
    if '추천' in prompt and '분석' not in prompt:
        response = '제품 추천을 원하는 문장입니다.'
    elif '분석' in prompt and '추천' not in prompt:
        response = '문제 해결을 위한 분석을 원하는 문장입니다.'
    else:
        messages = [
            SystemMessage(
                content="You are an assistant designed to classify user requests into one of two categories: (1) product recommendation or (2) problem-solving analysis. Your goal is to accurately determine whether the user is asking for product suggestions or seeking analytical help to resolve a problem based on the content and intent of the user's query."
            ),
            HumanMessage(
                content=f"{prompt}라는 문장이 제품 추천을 원하는 문장인지 문제 해결을 위한 분석을 원하는 문장인지 아니면 둘 다 아닌지 답변해줘"
            )
        ]
    response = llm.invoke(messages).content
    return response
    
def make_retrieveQA_form(contexts,question):
    prompt = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer." + contexts + "Question:" + question + "Helpful Answer: "
    return prompt

def solar_generate_text(client, prompt, adapter_id, max_new_tokens):
    answer = client.generate(prompt, adapter_id=adapter_id, max_new_tokens=max_new_tokens).generated_text
    return answer

def qa_chain_generate_text(qa_chain, prompt):
    if qa_chain =='solar_qa_chain':
        answer = solar_generate_text(client, prompt, adapter_id, max_new_tokens)
    else:
        answer = qa_chain.invoke({"query": prompt})['result']
    return answer

def generate_chat(llm, prompt):
    messages = [
        SystemMessage(
            content="You are an helpful assistant."
        ),
        HumanMessage(
            content=prompt
        )
    ]
    response = llm.invoke(prompt).content
    return response

class GraphState(TypedDict):
    question: str  # Prompt
    context: str  # Retrive result
    answer: str  # Answer
    relevance: str  # relevance check
    prompt_type: str # Analysis / Recommend
    product_type: str # Sup / Medication
    
def prompt_class(state : GraphState) -> GraphState:
    p_type = prompt_classification(base_llm, prompt)
    return GraphState(prompt_type = p_type)

def product_type(state : GraphState) -> GraphState:
    if tab == 'medi':
        return GraphState(product_type = 'medi')

def retrieve_medi_documents(question):
    retrieved_docs = ""
    info_retriever = kiwibm25_faiss_73_medi_info
    retrieved_info_docs = info_retriever.invoke(question)
    
    product_retriever = kiwibm25_faiss_73_medi_product
    retrieved_product_docs = product_retriever.invoke(retrieved_info_docs[0].page_content)
    # 검색된 문서를 context 키에 저장합니다.
    retrieved_info, medi_product = "", ""
    for info in retrieved_info_docs:
        retrieved_info += info.page_content + '\n'
    for m_product in retrieved_product_docs:
        medi_product += m_product.page_content + '\n'
    retrieved_docs = retrieved_info + "를 고려하여 제품을 추천하는데, \n " + medi_product + "이 정보들을 참고해서 Question 에 대한 답을 해줘 Question : " + question #### prompt
    return retrieved_docs 

def web_retriever(unknown_list, llm):
    retrieved_docs = []
    targets = ''
    for target in unknown_list:
        targets += target + ', '
        search_result = find_info_by_web_search(target+' 영양성분')
        search_result = generate_chat(base_llm, search_result + '이 내용을 참고해서 제품 이름과 영양성분을 정리해줘 정보가 명확하지 않을 경우 찾지 못했다고 말해주고 정리한 정보를 웹 검색으로 가져왔기 때문에 정확하지는 않다고도 말해줘')
        retrieved_docs.append(Document(page_content=search_result, metadata={"제품명": target}))
    print(targets) ## Unknown List
    return retrieved_docs, targets

def retrieve_document(state: GraphState) -> GraphState:
    if state['product_type'] == 'medi':
        retrieved_docs = retrieve_medi_documents(state["question"])
    else:
        retriever = kiwibm25_faiss_73_sup_product
        retrieved_docs = retriever.invoke(state["question"])
        targets = ''
        search_result_list = []
        if len(unknown_list) != 0:
            search_result_list, targets = web_retriever(unknown_list, base_llm)
            
    context_result = retrieved_docs + search_result_list
    n_question = targets + state['question']
    return GraphState(context = context_result, question = n_question) ###

#Solar Chatbot model을 사용하여 답변을 생성합니다.
def llm_answer(state: GraphState) -> GraphState:
    if state['product_type'] == 'medi':
        return GraphState(
        answer = generate_chat(base_llm, state['context']),
        context=state["context"],
        question=state["question"],
    )
    else:       
        contexts = ", ".join([state['context'][i].page_content for i in range(len(state['context']))])
        prompt = make_retrieveQA_form(contexts,state['question'])
        return GraphState(
            answer = qa_chain_generate_text(qa_chain, prompt),
            context=state["context"],
            question=state["question"],
        )
    
# Upstage Ground Checker로 관련성 체크를 실행합니다.
def relevance_check(state: GraphState) -> GraphState:
    # 관련성 체크를 실행합니다. 결과: grounded, notGrounded, notSure
    response = upstage_ground_checker.run(
        {"context": state["context"], "answer": state["answer"]}
    )
    return GraphState(
        relevance=response,
        context=state["context"],
        answer=state["answer"],
        question=state["question"],
    )

# 관련성 체크 결과를 반환합니다.
def is_relevant(state: GraphState) -> GraphState:
    if state["relevance"] == "grounded":
        return "관련성 O"
    elif state["relevance"] == "notGrounded":
        return "관련성 X"
    elif state["relevance"] == "notSure":
        return "확인불가"

solar_llm = Predibase(model="solar-1-mini-chat-240612", 
                predibase_api_key=os.environ["PREDIBASE_API_KEY"], 
                adapter_id="AIMedicine", 
                adapter_version=1,
                max_new_tokens = 4096)

gpt4o_llm = ChatOpenAI(model_name="gpt-4o", temperature=0.3)
solar_chat_llm = ChatUpstage(api_key=UPSTAGE_API_KEY, model="solar-1-mini-chat")

retriever = kiwibm25_faiss_73_sup_product

# RetrievalQA Chain 설정
gpt4o_qa_chain = RetrievalQA.from_chain_type(
    llm=gpt4o_llm,
    chain_type="stuff",
    retriever=retriever,
)

solar_qa_chain = RetrievalQA.from_chain_type(
    llm=solar_llm,
    chain_type="stuff",
    retriever=retriever,
)

workflow = StateGraph(GraphState)
#workflow.add_node("classification", prompt_classification)
workflow.add_node("product", product_type)
workflow.add_node("retrieve", retrieve_document)  # 에이전트 노드를 추가합니다.
workflow.add_node("llm_answer", llm_answer)  # 정보 검색 노드를 추가합니다.
workflow.add_node("relevance_check", relevance_check)  # 답변의 문서에 대한 관련성 체크 노드를 추가합니다.

#workflow.add_edge("classification","retrieve") # 프롬프트 타입 -> 검색
workflow.add_edge("product", "retrieve")
workflow.add_edge("retrieve", "llm_answer")  # 검색 -> 답변
workflow.add_edge("llm_answer", "relevance_check")  # 답변 -> 관련성 체크
workflow.add_conditional_edges(
    "relevance_check",  # 관련성 체크 노드에서 나온 결과를 is_relevant 함수에 전달합니다.
    is_relevant,
    {
        "관련성 O": END,  # 관련성이 있으면 종료합니다.
        "관련성 X": "retrieve",  # 관련성이 없으면 다시 답변을 생성합니다.
        "확인불가": "retrieve",  # 관련성 체크 결과가 모호하다면 다시 답변을 생성합니다.
    },
)

workflow.set_entry_point("product")

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# recursion_limit: 최대 반복 횟수, thread_id: 실행 ID (구분용)
config = RunnableConfig(recursion_limit=5, configurable={"thread_id": "SELF-RAG"})

#query = "멀티비타민 올인원, 밀크씨슬"
#prompt = f"저의 성별은 남성, 나이는 성인 (19세 이상)입니다. 제가 지금 먹고 있는 영양제 제품들은 {query} 이고 이렇게 먹으면 적절한지 권장 섭취량과 현재 복용중인 영양성분을 비교해서 확인해주세요."

#prompt = "두통이 있는데 어떤 약을 먹는게 좋을까?"
query = "루테인 지아잔틴 아스타잔틴, 비타앤 히알루론산 피치맛"
prompt = f"{query} 제품들을(를) 같이 먹고 있는데 적합하게 먹고 있어? 권장섭취량을 참고해서 지금 먹고 있는 영양제가 권장량에 작합한지 비교해줘"
pb_token = pb(api_token=PREDIBASE_API_KEY)
client = pb_token.deployments.client("solar-1-mini-chat-240612")
adapter_id, max_new_tokens ="AIMedicine/1", 2000

tab = 'sup' # sup or medi
unknown_list = []
inputs = GraphState(question=prompt, product_type=tab)
#qa_chain = solar_qa_chain  # solar_qa_chain or gpt4o_qa_chain
qa_chain = gpt4o_qa_chain
base_llm = gpt4o_llm
output = app.invoke(inputs, config=config)

print("Question: \t", output["question"])
print("Answer: \t", output["answer"])
print("Relevance: \t", output["relevance"])

WARN: Currently installed SDK is outdated. This can lead to bugs or unexpected behavior. Consider upgrading to the 
latest version. Installed: 2024.8.4 Latest: 2024.10.1.

WARN: Currently installed SDK is outdated. This can lead to bugs or unexpected behavior. Consider upgrading to the 
latest version. Installed: 2024.8.4 Latest: 2024.10.1.

Connected to Predibase as User(id=b5555c02-4e64-48dd-b993-2f03a91a605d, username=sehoonpark@hanyang.ac.kr)

5
Question: 	 루테인 지아잔틴 아스타잔틴, 비타앤 히알루론산 피치맛 제품들을(를) 같이 먹고 있는데 적합하게 먹고 있어? 권장섭취량을 참고해서 지금 먹고 있는 영양제가 권장량에 작합한지 비교해줘
Answer: 	 루테인 지아잔틴 아스타잔틴과 비타앤 히알루론산 피치맛 제품을 함께 섭취하는 것이 적합한지 확인하기 위해서는 각 제품의 영양성분과 권장 섭취량을 비교해야 합니다. 

1. **루테인 지아잔틴 아스타잔틴**:
   - 비타민A: 210ug
   - 아스타잔틴: 6mg
   - 루테인지아잔틴복합추출물: 20mg

2. **비타앤 히알루론산 피치맛**:
   - 비타민C: 300mg
   - 히알루론산: 120mg

각 제품의 영양성분은 서로 중복되지 않으며, 특정 성분의 과다 섭취로 인한 부작용이 명시되어 있지 않습니다. 그러나, 비타민A와 비타민C의 경우 일반적인 권장 섭취량을 초과하지 않는지 확인해야 합니다. 

- **비타민A**: 성인 남성의 경우 하루 권장량은 약 900ug, 여성의 경우 약 700ug입니다. 루테인 지아잔틴 아스타잔틴의 비타민A 함량은 210ug로, 권장량을 초과하지 않습니다.
- **비타민C**: 성인의 경우 하루 권장량은 약 75-90mg입니다. 비타앤 히알루론산 피치맛의 비타민C 함량은 300mg로, 권장량을 초과할 수 있으므로 주의가 필요합니다.

따라서, 두 제품을 함께 섭취하는 것은 일반적으로 문제가 없으나, 비타민C의 경우 권장량을 초과할 수 있으므로 주의가 필요합니다. 개인의 건강 상태에 따라 다를 수 있으니, 의사나 영양사와 상담하는 것이 좋습니다.
Relevance: 	 grounded
